In [1]:
import igraph as ig
import numpy as np
import sys,os

# Load the graphs

In [17]:
sys.path.append('..')
if "wrappers.spektral_wrapper" in sys.modules:
    del sys.modules["wrappers.spektral_wrapper"]

from wrappers.spektral_wrapper import MyGMLDataset, MyTUDataset

In [18]:
#ataset = MyGMLDataset('YENDIK', '../../GraphML/datasets')
dataset2 = MyGMLDataset('Mutag', '../../../CDS-GROUP/GE METANET/', upto=5)

100%|██████████| 5/5 [00:00<00:00, 319.47it/s]

(5, 2)
Successfully loaded Mutag.


In [21]:
dataset2[0].x.shape, dataset2[0].e.shape,dataset2[0].a.shape,dataset2.n_labels,dataset2.n_node_features
type(dataset2[0].a),dataset2[0].a, type(dataset2[0].a.data)

(numpy.ndarray,
 array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0,

In [8]:
dataset[0].x.shape, dataset[0].e.shape,dataset[0].a.shape,dataset.n_labels,dataset.n_node_features
type(dataset[0].a),dataset[0].a.data, type(dataset[0].a.data)

(scipy.sparse.csr.csr_matrix,
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.]),
 numpy.ndarray)

In [3]:
dataset = MyTUDataset('MUTAG', 'MUTAG')

Successfully loaded MUTAG.


In [41]:
dataset2[0].y.shape, dataset[0].y.shape

((2,), (3,))

# Apply GIN-0 

In [9]:
from time import time
import tensorflow as tf
import tqdm.notebook as tq
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import TUDataset
from spektral.layers import GINConv, GlobalSumPool, GlobalAvgPool
from spektral.data import DisjointLoader
from spektral.datasets import TUDataset
from spektral.layers import GINConv, GlobalSumPool, GlobalAvgPool

import tqdm as tq

################################################################################
# Config
################################################################################
#@title Parameters { form-width: "30%" }
learning_rate = 0.001  #@param {type:"number"}
epochs = 200 #@param {type:"slider", min:0, max:500, step:20}
channels = 64 #@param {type:"slider", min:16, max:128, step:16}
batch_size = 1  #@param {type:"slider", min:1, max:64, step:1}
layers = 3  #@param {type:"slider", min:1, max:5, step:1}
folds = 5  #@param {type:"slider", min:1, max:10, step:1}
verbose = True #@param {type:"boolean"}
seed = 42 #@param {type:"number"}

# Parameters
F = dataset.n_node_features  # Dimension of node features
n_out = dataset.n_labels  # Dimension of the target
print("Node features", F)
print(dataset[0].x)
#sys.exit(0)
################################################################################
# Build model
################################################################################
class GIN0(Model):
    def __init__(self, channels, n_layers):
        super().__init__()
        self.conv1 = GINConv(channels, epsilon=0, mlp_hidden=[channels, channels])
        self.convs = []
        for _ in range(1, n_layers):
            self.convs.append(
                GINConv(channels, epsilon=0, mlp_hidden=[channels, channels])
            )
        self.pool = GlobalAvgPool()
        self.dense1 = Dense(channels, activation="relu")
        self.dropout = Dropout(0.5)
        self.dense2 = Dense(n_out, activation="softmax")

    def call(self, inputs):
        x, a, = inputs[0:2]
        i = inputs[-1]
        #x, a, i = inputs
        x = self.conv1([x, a])
        for conv in self.convs:
            x = conv([x, a])
        x = self.pool([x, i])
        x = self.dense1(x)
        x = self.dropout(x)
        return self.dense2(x)

start = time()

# Cross Validation loop
from sklearn.model_selection import StratifiedKFold
targets = [g.y.dot(1 << np.arange(g.y.size)[::-1]) for g in dataset]
sp = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
results = []
for idx_tr, idx_te in tq.tqdm(list(sp.split(dataset, targets)), desc="fold: "):
  dataset_tr, dataset_te = dataset[idx_tr], dataset[idx_te]

  loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
  loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)

################################################################################
# Fit model
################################################################################
  @tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
  def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc = tf.reduce_mean(categorical_accuracy(target, predictions))
    return loss, acc
    
  # Build model
  model = GIN0(channels, layers)
  optimizer = Adam(learning_rate)
  loss_fn = CategoricalCrossentropy()
  @tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
  def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc = tf.reduce_mean(categorical_accuracy(target, predictions))
    return loss, acc
  # Build model
  model = GIN0(channels, layers)
  optimizer = Adam(learning_rate)
  loss_fn = CategoricalCrossentropy()

  epoch = step = 0
  tresults = []
  for batch in loader_tr:
    step += 1
    loss, acc = train_step(*batch)
    tresults.append((loss, acc))
    if step == loader_tr.steps_per_epoch:
        step = 0
        epoch += 1
        if verbose: print("Ep. {} - Loss: {}. Acc: {}".format(epoch, *np.mean(tresults, 0)))
        tresults = []

################################################################################
# Evaluate model
################################################################################
  for batch in loader_te:
    inputs, target = batch
    predictions = model(inputs, training=False)
    results.append(
        (
            loss_fn(target, predictions),
            tf.reduce_mean(categorical_accuracy(target, predictions)),
        )
    )
  if verbose: print("Done. Test loss: {}. Test acc: {}".format(*np.mean(results, 0)))
# Timing
temp = time() - start
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes
expired = '%d:%d:%d' %(hours,minutes,seconds)
print("Done. Test loss: {}. Test acc: {}".format(*np.mean(results, 0)))

method = 'GIN'
from datetime import datetime
import pandas as pd
path = f'{method}_results.csv'
if not os.path.exists(path):
  dfres = pd.DataFrame(columns=['dataset', 'loss', 'acc', 'folds', 'seed', 'lr', 'epochs', 'batch_size', 'layers', 'channels', 'date', 'elapsed'])
  dfres.to_csv(path, index=False)
dfres = pd.read_csv(path)
dfres = dfres.append({'dataset': dataname,
                      'loss': np.mean(results, 0)[0],
                      'acc' : np.mean(results, 0)[1],
                      'folds' : folds,
                      'seed' : seed,
                      'lr' : learning_rate, 'epochs' : epochs, 'batch_size' : batch_size, 'layers': layers, 'channels' : channels,
                      'date': datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),
                      'elapsed': expired
                      }, ignore_index=True)
dfres.to_csv(path, index=False)
dfres

fold:   0%|          | 0/5 [00:00<?, ?it/s]

Node features 82
[[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
Ep. 1 - Loss: 1.042484164237976. Acc: 0.4686192572116852
Ep. 2 - Loss: 1.0100605487823486. Acc: 0.5313807725906372
Ep. 3 - Loss: 1.0226746797561646. Acc: 0.5271966457366943
Ep. 4 - Loss: 1.0115495920181274. Acc: 0.5271966457366943
Ep. 5 - Loss: 1.0093621015548706. Acc: 0.5355648398399353
Ep. 6 - Loss: 1.0282316207885742. Acc: 0.5313807725906372
Ep. 7 - Loss: 1.013115406036377. Acc: 0.5313807725906372
Ep. 8 - Loss: 1.0087506771087646. Acc: 0.5313807725906372
Ep. 9 - Loss: 1.0152674913406372. Acc: 0.5313807725906372
Ep. 10 - Loss: 0.9925516843795776. Acc: 0.5313807725906372
Ep. 11 - Loss: 1.0069535970687866. Acc: 0.5313807725906372
Ep. 12 - Loss: 1.0134238004684448. Acc: 0.5313807725906372
Ep. 13 - Loss: 1.0238357782363892. Acc: 0.5313807725906372
Ep. 14 - Loss: 1.001293420791626. Acc: 0.5313807725906372
Ep. 15 - Loss

fold:  20%|██        | 1/5 [09:47<39:08, 587.10s/it]

Done. Test loss: 0.9950946569442749. Test acc: 0.5333333611488342
Ep. 1 - Loss: 1.0374271869659424. Acc: 0.5188284516334534
Ep. 2 - Loss: 1.0112583637237549. Acc: 0.5230125784873962
Ep. 3 - Loss: 1.0165907144546509. Acc: 0.5313807725906372
Ep. 4 - Loss: 1.0059751272201538. Acc: 0.5313807725906372
Ep. 5 - Loss: 1.0087835788726807. Acc: 0.5313807725906372
Ep. 6 - Loss: 1.0250953435897827. Acc: 0.5313807725906372
Ep. 7 - Loss: 1.0175304412841797. Acc: 0.5313807725906372
Ep. 8 - Loss: 1.0130006074905396. Acc: 0.5313807725906372
Ep. 9 - Loss: 1.002454400062561. Acc: 0.5313807725906372
Ep. 10 - Loss: 1.0103925466537476. Acc: 0.5313807725906372
Ep. 11 - Loss: 1.0048426389694214. Acc: 0.5313807725906372
Ep. 12 - Loss: 1.0143581628799438. Acc: 0.5313807725906372
Ep. 13 - Loss: 1.0127204656600952. Acc: 0.5313807725906372
Ep. 14 - Loss: 1.0107499361038208. Acc: 0.5313807725906372
Ep. 15 - Loss: 1.005923867225647. Acc: 0.5313807725906372
Ep. 16 - Loss: 1.0109561681747437. Acc: 0.5313807725906372
E

fold:  40%|████      | 2/5 [19:07<28:33, 571.15s/it]

...

fold:  80%|████████  | 4/5 [38:23<09:36, 576.42s/it]

Done. Test loss: 0.9951061010360718. Test acc: 0.5333333611488342
Ep. 1 - Loss: 1.0479801893234253. Acc: 0.5041666626930237
Ep. 2 - Loss: 1.0320168733596802. Acc: 0.5333333611488342
Ep. 3 - Loss: 1.0134187936782837. Acc: 0.5333333611488342
Ep. 4 - Loss: 1.0159051418304443. Acc: 0.5333333611488342
Ep. 5 - Loss: 1.0126702785491943. Acc: 0.5333333611488342
Ep. 6 - Loss: 1.0251668691635132. Acc: 0.5333333611488342
Ep. 7 - Loss: 1.0114772319793701. Acc: 0.5333333611488342
Ep. 8 - Loss: 1.0178523063659668. Acc: 0.5333333611488342
Ep. 9 - Loss: 1.0004714727401733. Acc: 0.5333333611488342
Ep. 10 - Loss: 1.010414719581604. Acc: 0.5333333611488342
Ep. 11 - Loss: 1.010692834854126. Acc: 0.5333333611488342
Ep. 12 - Loss: 1.0047698020935059. Acc: 0.5333333611488342
Ep. 13 - Loss: 1.0048631429672241. Acc: 0.5333333611488342
Ep. 14 - Loss: 1.0112051963806152. Acc: 0.5333333611488342
Ep. 15 - Loss: 1.0020415782928467. Acc: 0.5333333611488342
Ep. 16 - Loss: 1.0056232213974. Acc: 0.5333333611488342
Ep. 

fold: 100%|██████████| 5/5 [47:52<00:00, 574.51s/it]

Done. Test loss: 0.9963248372077942. Test acc: 0.5317725539207458
Done. Test loss: 0.9963248372077942. Test acc: 0.5317725539207458


NameError: name 'dataname' is not defined